# Model Tuning, Interpretation and Deployment

## Overview

This tutorial covers essential aspects of the machine learning workflow after initial model building, focusing on model interpretation techniques, tuning approaches, and deployment considerations. We'll explore how to make machine learning models more interpretable, optimize their performance through tuning, and prepare them for real-world deployment.

## Learning Objectives

- Understand the importance of model interpretation in machine learning
  - Learn how interpretability benefits analytics teams and stakeholders
  - Recognize how interpretability bridges technical and business understanding
- Master techniques for model tuning and optimization
- Learn best practices for model deployment
- Develop skills to explain model decisions to non-technical stakeholders

### Tasks to complete

- Implement model interpretation techniques
- Perform model tuning exercises
- Practice model deployment steps
- Create interpretability visualizations

## Prerequisites

- A working Python environment and familiarity with Python
- Basic understanding of machine learning concepts
- Familiarity with pandas and numpy libraries
- Knowledge of basic statistical concepts


## Get Started

- Please select kernel "conda_tensorflow2_p310" from SageMaker notebook instance.


### Import libraries


In [ ]:
# Install the 'lime' and 'shap' Python packages using pip.
# These packages are commonly used for model interpretability and explainability in machine learning.
%pip install lime shap

In [ ]:
# Import the warnings module to handle warnings
import warnings

# Import joblib for efficient saving and loading of Python objects
import joblib

# Import NumPy for numerical operations, especially for handling arrays
import numpy as np

# Import pandas for data manipulation and analysis, particularly for working with DataFrames
import pandas as pd

# Import SciPy for scientific and technical computing, including statistical functions
import scipy

# Import SHAP library for explaining the output of machine learning models
import shap  # Import SHAP for explanation

# Import LimeTabularExplainer from the lime library for explaining tabular data predictions
from lime.lime_tabular import LimeTabularExplainer  # Import LIME for explanation

# Import linear_model module from scikit-learn for linear models like Logistic Regression
from sklearn import linear_model, metrics

# Import the load_breast_cancer dataset from scikit-learn for demonstration purposes
from sklearn.datasets import load_breast_cancer

# Import GridSearchCV and RandomizedSearchCV for hyperparameter tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

# Import SVC (Support Vector Classifier) from scikit-learn for classification tasks
from sklearn.svm import SVC

# Model Tunning, Interpretation and Deployment

Adapted from Dipanjan Sarkar et al. 2018. [Practical Machine Learning with Python](https://link.springer.com/book/10.1007/978-1-4842-3207-1).


In this tutorial, we will learn:

- How to tune the hyperparameters of Machine Learning algorithms
- How to interpret models using open source frameworks
- How to persist and deploy the developed models


## Model tuning

Model tuning is one of the
most important concepts of Machine Learning and it does require some knowledge of the underlying math
and logic of the algorithm in focus. In this tutorial, we will delve deeper into the models that we are targeting, look at the knobs
that can be tuned and set to extract the best performance out of any given models. This process of iterative
experimentation with dataset, model parameters, and features is the very core of the model tuning process.


### Build and Evaluate Default Model

We will use Wisconsin Breast Cancer Dataset as an example. We first split the breast cancer datast variables X and y into train and test datasets and build an SVM model with default parameters. Then we will evaluate its performance on the test dataset.


In [ ]:
# Load Wisconsin Breast Cancer Dataset
from sklearn.datasets import load_breast_cancer

# load data
bc = load_breast_cancer()

# Extract the feature data (input features) from the dataset
X = bc.data
# Extract the target data (labels or output) from the dataset
y = bc.target

# Print the shape of the feature data (number of samples and features) and the names of the features
print(X.shape, bc.feature_names)

In [ ]:
# Utility functions for model evaluation

# Get model performance evaluation matrics
def get_metrics(true_labels, predicted_labels):
    # Print the accuracy score, rounded to 4 decimal places, by comparing true labels to predicted labels.
    print(
        "Accuracy:", np.round(metrics.accuracy_score(true_labels, predicted_labels), 4)
    )
    # Print the precision score, rounded to 4 decimal places, calculated with weighted averaging for multi-class, by comparing true labels to predicted labels.
    print(
        "Precision:",
        np.round(
            metrics.precision_score(true_labels, predicted_labels, average="weighted"),
            4,
        ),
    )
    # Print the recall score, rounded to 4 decimal places, calculated with weighted averaging for multi-class, by comparing true labels to predicted labels.
    print(
        "Recall:",
        np.round(
            metrics.recall_score(true_labels, predicted_labels, average="weighted"), 4
        ),
    )
    # Print the F1 score, rounded to 4 decimal places, calculated with weighted averaging for multi-class, by comparing true labels to predicted labels.
    print(
        "F1 Score:",
        np.round(
            metrics.f1_score(true_labels, predicted_labels, average="weighted"), 4
        ),
    )


# Show the classification report
def display_classification_report(true_labels, predicted_labels, classes=[1, 0]):
    # Build a text report showing the main classification metrics
    # This line calculates and stores the classification report as a string.
    # It uses the `classification_report` function from the `metrics` module (likely scikit-learn).
    # `y_true=true_labels`:  Specifies the true class labels.
    # `y_pred=predicted_labels`: Specifies the predicted class labels from the model.
    # `labels=classes`:  Specifies the classes to be included in the report, here defaulting to [1, 0].
    report = metrics.classification_report(
        y_true=true_labels, y_pred=predicted_labels, labels=classes
    )
    # Print the classification report to the console.
    # This will display the precision, recall, f1-score, and support for each class,
    # as well as overall accuracy and macro/weighted averages.
    print(report)


# Show the confusion matrix
def display_confusion_matrix(true_labels, predicted_labels, classes=[1, 0]):
    # Determine the total number of classes from the classes list.
    total_classes = len(classes)
    # Define levels for MultiIndex labels in the DataFrame, used for formatting the confusion matrix.
    level_labels = [total_classes * [0], list(range(total_classes))]
    # Compute the confusion matrix using scikit-learn's metrics.confusion_matrix function.
    cm = metrics.confusion_matrix(
        y_true=true_labels, y_pred=predicted_labels, labels=classes
    )
    # Create a Pandas DataFrame to display the confusion matrix in a structured format.
    cm_frame = pd.DataFrame(
        data=cm,
        # Set column names for the DataFrame using MultiIndex to represent 'Predicted' and class labels.
        columns=pd.MultiIndex(levels=[["Predicted:"], classes], codes=level_labels),
        # Set index names for the DataFrame using MultiIndex to represent 'Actual' and class labels.
        index=pd.MultiIndex(levels=[["Actual:"], classes], codes=level_labels),
    )
    # Print the confusion matrix DataFrame to the console.
    print(cm_frame)


# Show the model performace matrics
def display_model_performance_metrics(true_labels, predicted_labels, classes=[1, 0]):
    # Prints a header for model performance metrics
    print("Model Performance metrics:")
    # Prints a separator line for visual clarity
    print("-" * 30)
    # Calls the function to calculate and print performance metrics
    get_metrics(true_labels=true_labels, predicted_labels=predicted_labels)
    # Prints a newline and header for the classification report
    print("\nModel Classification report:")
    # Prints a separator line for visual clarity
    print("-" * 30)
    # Calls the function to display the classification report
    display_classification_report(
        true_labels=true_labels, predicted_labels=predicted_labels, classes=classes
    )
    # Prints a newline and header for the confusion matrix
    print("\nPrediction Confusion Matrix:")
    # Prints a separator line for visual clarity
    print("-" * 30)
    # Calls the function to display the confusion matrix
    display_confusion_matrix(
        true_labels=true_labels, predicted_labels=predicted_labels, classes=classes
    )

In [ ]:
# Prepare datasets for training and testing, splitting the data into training and test sets.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Build a default Support Vector Machine (SVM) model.
# Initialize a C-Support Vector Classification model with a fixed random state for reproducibility.
def_svc = SVC(random_state=42)
# Train the default SVM model using the training data (features X_train and labels y_train).
def_svc.fit(X_train, y_train)

# Predict labels for the test dataset using the trained default SVM model.
def_y_pred = def_svc.predict(X_test)
# Print a header to indicate the performance metrics for the default model.
print("Default Model Stats:")
# Display and print the performance metrics of the default model using the test labels (y_test) and the predicted labels (def_y_pred).
# The metrics will be displayed for classes 0 and 1.
display_model_performance_metrics(
    true_labels=y_test, predicted_labels=def_y_pred, classes=[0, 1]
)

### Tune Model with Grid Search

Since we have chosen a SVM model, we specify some hyperparameters specific
to it, which includes the Regularization parameter C (deals with the margin parameter in SVM), the kernel function (used
for transforming data into a higher dimensional feature space) and gamma (determines the influence a
single training data point has). There are a lot of other hyperparameters to tune, which you can check out [here](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) for further details.

We will build a grid by supplying some pre-set values. The next choice is selecting the score or metric we want
to maximize here we have chosen to maximize accuracy of the model. Once that is done, we will be using
five-fold cross-validation to build multiple models over this grid and evaluate them to get the best model.

(This step should take about three minutes to complete.)


In [ ]:
# Define the grid of hyperparameters to search over for the SVM model.
grid_parameters = {
    "kernel": [
        "linear",
        "rbf",
    ],  # Kernels to try: linear and radial basis function (rbf).
    "gamma": [1e-3, 1e-4],  # Gamma values to try for rbf kernel (kernel coefficient).
    "C": [1, 10, 50, 100],  # C values to try (regularization parameter).
}

# Indicate that hyperparameter tuning is starting, specifically for optimizing accuracy.
print("# Tuning hyper-parameters for accuracy\n")

# Initialize GridSearchCV for hyperparameter tuning of an SVC classifier.
# SVC(random_state=42): Creates an SVC classifier with a fixed random state for reproducibility.
# grid_parameters: The parameter grid defined above to search through.
# cv=5: Perform 5-fold cross-validation.
# scoring="accuracy": Evaluate models based on accuracy.
clf = GridSearchCV(SVC(random_state=42), grid_parameters, cv=5, scoring="accuracy")
# Fit the GridSearchCV object to the training data (X_train, y_train) to find the best hyperparameter combination.
clf.fit(X_train, y_train)

# Display the accuracy scores obtained for each hyperparameter combination during cross-validation.
print("Grid scores for all the models based on CV:\n")
# Extract the mean test scores from the GridSearchCV results. These are the average accuracy scores across the cross-validation folds for each parameter combination.
means = clf.cv_results_["mean_test_score"]
# Extract the standard deviation of the test scores from the GridSearchCV results. This indicates the variability of the accuracy scores across the cross-validation folds for each parameter combination.
stds = clf.cv_results_["std_test_score"]
# Iterate through the mean scores, standard deviations, and parameter combinations to print the results for each model.
for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
    # Print the mean accuracy score and its standard deviation (multiplied by 2 to represent approximately 95% confidence interval) for each parameter setting.
    print("%0.5f (+/-%0.05f) for %r" % (mean, std * 2, params))

# Output the best hyperparameter combination found by GridSearchCV on the development set (which is the training set in this case, due to cross-validation).
print("\nBest parameters set found on development set:", clf.best_params_)
# Output the best mean cross-validation score (accuracy) achieved with the best hyperparameter combination. This is an estimate of the model's performance on unseen data.
print("Best model validation accuracy:", clf.best_score_)

We can see the best model parameters were obtained
based on cross-validation accuracy and we get a pretty awesome validation accuracy of 96%.


### Evaluate Grid Search Tuned Model

Let’s take this
optimized and tuned model and put it to the test on our test data!


In [ ]:
# Retrieves the best estimator found by GridSearchCV (or similar hyperparameter tuning process).
gs_best = clf.best_estimator_
# Uses the best estimator to make predictions on the test set (X_test).
tuned_y_pred = gs_best.predict(X_test)

# Prints a header to indicate the performance of the tuned model.
print("\n\nTuned Model Stats:")
# Calls a function to display performance metrics for the tuned model.
display_model_performance_metrics(
    true_labels=y_test, predicted_labels=tuned_y_pred, classes=[0, 1]
)

Our model gives an overall F1 Score and model
accuracy of 97% on the test dataset too. This should give you a clear indication of
the power of hyperparameter tuning! This scheme of things can be extended for different models and their
respective hyperparameters. We can also play around with the evaluation measure we want to optimize.
The scikit-learn framework provides us with different values that we can optimize. Some of them are
adjusted_rand_score, average_precision, f1, average_recall, and so on.


### Tune Model with Randomized Search

Grid search suffers from some major shortcomings, the most important one being
the limitation of manually specifying the grid. This brings a human element into a process that could benefit
from a purely automatic mechanism.

Randomized parameter search is a modification to the traditional grid search. It takes input for
grid elements as in normal grid search but it can also take distributions as input. For example consider
the parameter gamma whose values we supplied explicitly in the last section instead we can supply a
distribution from which to sample gamma.

The efficacy of randomized parameter search is based on the
proven (empirically and mathematically) result that the hyperparameter optimization functions normally
have low dimensionality and the effect of certain parameters are more than others.

We control the number
of times we want to do the random parameter sampling by specifying the number of iterations we want to
run (n_iter). Normally a higher number of iterations mean a more granular parameter search but higher
computation time.

(This step should take about five minutes to complete.)


In [ ]:
# Define the parameter grid for RandomizedSearchCV.
param_grid = {
    # Define 'C' hyperparameter to be sampled from an exponential distribution with scale=10.
    "C": scipy.stats.expon(scale=10),
    # Define 'gamma' hyperparameter to be sampled from an exponential distribution with scale=0.1.
    "gamma": scipy.stats.expon(scale=0.1),
    # Define 'kernel' hyperparameter to choose from 'rbf' or 'linear'.
    "kernel": ["rbf", "linear"],
}

# Initialize RandomizedSearchCV for hyperparameter tuning of SVC.
random_search = RandomizedSearchCV(
    # Use SVC classifier with a fixed random state for reproducibility.
    SVC(random_state=42),
    # Specify the parameter distributions to sample from.
    param_distributions=param_grid,
    # Set the number of iterations for random parameter combinations to 50.
    n_iter=50,
    # Use 5-fold cross-validation.
    cv=5,
)
# Fit the RandomizedSearchCV model to the training data (X_train, y_train).
random_search.fit(X_train, y_train)
# Print a header for the grid scores from cross-validation.
print("Grid scores for all the models based on CV:\n")
# Extract the mean test scores from the RandomizedSearchCV results.
means = random_search.cv_results_["mean_test_score"]
# Extract the standard deviation of the test scores from the RandomizedSearchCV results.
stds = random_search.cv_results_["std_test_score"]
# Iterate through the mean scores, standard deviations, and parameter sets from the cross-validation.
for mean, std, params in zip(means, stds, random_search.cv_results_["params"]):
    # Print the mean score, 95% confidence interval (std * 2), and the corresponding parameter set for each model.
    print("%0.5f (+/-%0.05f) for %r" % (mean, std * 2, params))
# Print a header for the best parameter set found by RandomizedSearchCV.
print("\nBest parameters set found on development set:", random_search.best_params_)
# Print the best model's validation accuracy (mean cross-validation score for the best parameter set).
print("Best model validation accuracy:", random_search.best_score_)

## Evaluate Randomized Search Tuned Model

Get the best model, predict and evaluate performance


In [ ]:
# Retrieve the best estimator found by RandomizedSearchCV.
rs_best = random_search.best_estimator_
# Use the best estimator to predict labels for the test dataset (X_test).
rs_y_pred = rs_best.predict(X_test)
# Evaluate the performance of the best estimator using a function called get_metrics,
# comparing the true labels (y_test) with the predicted labels (rs_y_pred).
get_metrics(true_labels=y_test, predicted_labels=rs_y_pred)

We are getting the values of parameter C and gamma from an exponential distribution
and we are controlling the number of iterations of model search by the parameter n_iter. While the overall
model performance is similar to grid search, the intent is to be aware of the different strategies in model tuning.


## Model Interpretation

The ability to interpret Machine Learning models in an easy to understand way will benefit not only analytics teams but also key stakeholders in trying to explain how models really work.

Some Machine Learning models use interpretable algorithms, for example a decision tree will give you
the importance of all the variables as an output. Unfortunately, this
can’t be said for a lot of models, especially for the ones who have no notion of variable importance.

The lack of understanding of the complex nature
of Machine Learned decision policies makes predictive models to be still viewed as black boxes. Model
interpretations can help a data scientist and an end user in a variety of ways.

- It will help bridge the gap that
  often exists between the technology teams and the business. For example, it can help identify the reason
  why a particular prediction is being made and it can be verified using the domain knowledge of the end
  user by leveraging that easy to understand interpretation.
- It can also help the data scientists understand the
  interactions among features that can lead to better feature engineering and enhanced performance.
- It can
  also help in model comparisons and explaining the results better to the business stakeholders.


### Understanding LIME and SHAP

LIME (Local Interpretable Model-agnostic Explanations) and SHAP (SHapley Additive exPlanations) are two powerful tools designed to explain machine learning model predictions in an interpretable manner, making complex models more transparent. LIME works by approximating a machine learning model with a simple, interpretable model (such as a linear regression) around the instance being predicted. It perturbs the input data, generates predictions, and learns a locally faithful model that explains the specific prediction for that instance. This method provides insights into how a model makes decisions for individual instances. SHAP, on the other hand, is grounded in game theory and computes the contribution of each feature to the model’s prediction by distributing the prediction's total impact fairly across all features. SHAP values are additive, and the method provides both local and global interpretability, making it highly effective for understanding the overall impact of features on model predictions. While LIME focuses on local explanations, SHAP excels in providing consistent global feature importance, making both techniques complementary for enhancing model transparency and trustworthiness.


In [ ]:
# Import the warnings module to handle and filter warnings.
import warnings

# Filter all warnings to be ignored. This is often used to suppress less important warning messages for cleaner output.
warnings.filterwarnings("ignore")

# Import the LogisticRegression class from the linear_model module of the sklearn library.
# This class will be used to create a logistic regression model for classification.
from sklearn import linear_model

# Initialize a Logistic Regression model object.
# This creates an instance of the LogisticRegression classifier with default parameters.
logistic = linear_model.LogisticRegression()
# Train the Logistic Regression model using the training data.
# X_train is the feature matrix of the training data, and y_train is the target variable (labels) for the training data.
# The fit() method learns the relationship between features and target variable from the training data.
logistic.fit(X_train, y_train)

In [ ]:
# SHAP Explanation (instead of Skater's Interpretation)
# Uses kmeans clustering to create a background dataset from the training data (X_train) for KernelExplainer.
background = shap.kmeans(X_train, 50)  # Summarize background using 50 clusters
# Initializes a KernelExplainer object. KernelExplainer is model-agnostic and approximates SHAP values for any prediction function.
# It uses the logistic.predict_proba function (likely from a trained logistic regression model) to explain predictions.
# The 'background' dataset is used to estimate expected values in the SHAP calculation, improving efficiency.
explainer = shap.KernelExplainer(
    logistic.predict_proba, background
)  # Use KernelExplainer
# Calculates SHAP values for the test dataset (X_test).
# SHAP values quantify the contribution of each feature to the prediction for each instance in X_test, based on the KernelExplainer and the model's predict_proba function.
shap_values = explainer.shap_values(X_test)

In [ ]:
# Plot SHAP summary plot
# This line generates a SHAP summary plot, which is a visualization to understand feature importance and their impact on the model output.
# shap_values: The SHAP values calculated for the test dataset (X_test). These values represent the contribution of each feature to each individual prediction.
# X_test: The test dataset used for prediction. This is needed to show the actual feature values in the summary plot.
# feature_names=bc.feature_names:  Specifies the names of the features. It's assumed 'bc.feature_names' contains a list of feature names corresponding to the columns in X_test, likely from a dataset object 'bc'.
shap.summary_plot(shap_values, X_test, feature_names=bc.feature_names)

### Key Takeaways from the Plot

- The SHAP interaction values are centered around 0, meaning no strong interaction effects dominate.
- Mean radius and mean texture seem to have moderate interactions, with red and blue points somewhat spread out.
- If you see larger deviations from 0 in any interaction, it indicates that the combination of those features significantly impacts predictions (either increasing or decreasing the model output more than their individual contributions).


### Explaining Predictions

Let’s try to interpret some actual predictions now. We will predict two data points, one not
having cancer (label 1) and one having cancer (label 0), and try to interpret the prediction making process.


In [ ]:
# Initialize LimeTabularExplainer for explaining tabular data predictions.
lime_explainer = LimeTabularExplainer(
    X_train,  # Training data (numpy array or pandas DataFrame) used to understand the feature ranges and distributions.
    feature_names=bc.feature_names,  # List of feature names corresponding to the columns in X_train.
    discretize_continuous=True,  # Whether to discretize continuous features. Set to True for tabular data.
    class_names=["0", "1"],  # List of class names or labels for the target variable.
)

In [ ]:
# Generate explanation for an individual prediction from the test set using LIME.
lime_exp = lime_explainer.explain_instance(X_test[0], logistic.predict_proba)
# Display the LIME explanation in the notebook for visual interpretation.
lime_exp.show_in_notebook()

### Key Takeaways from the LIME Explanation above

1. Prediction Probabilities

   - The model predicts 87% (0.87) probability for Malignant (1).
   - The probability for Benign (0) is only 13% (0.13).
   - This means the model strongly believes the tumor is malignant.

2. Feature Contributions

   - Features supporting Malignant (1) are in orange (positive contribution to malignancy).
   - Features supporting Benign (0) are in blue (negative contribution to malignancy, pushing toward benign).
   - The most important malignant indicators are:
     - Worst perimeter (96.05)
     - Worst area (677.90)
     - Mean area (481.90)
     - Area error (30.29)
   - The most important benign indicators (blue) are:
     - Worst radius (14.97)
     - Mean perimeter (81.09)
     - Mean radius (12.47)

3. Feature Value Ranges
   - The middle section lists decision splits from the model (e.g., “84.54 < worst perimeter <= 97.75” means a higher perimeter increases malignancy probability).
   - Larger values for worst perimeter, worst area, and mean area strongly push the prediction toward malignancy.

#### Final Interpretation

Even though some features (blue) support the benign classification, the dominant malignant-supporting features outweigh them. Therefore, the model predicts Malignant (1) with high confidence (87%).


Let’s run a similar interpretation on a data point with malignant
cancer.


In [ ]:
# Explain the prediction for a single instance (the second instance) from the test set (X_test[1])
lime_exp = lime_explainer.explain_instance(X_test[1], logistic.predict_proba)
# Display the Lime explanation in the notebook for visual interpretation.
lime_exp.show_in_notebook()

### Key Takeaways from the LIME Explanation above

1. Prediction Probabilities
   - The model is 100% certain this is a benign tumor (0.00 probability for malignant (1)).
   - The blue bar (0) is fully filled, meaning all supporting features push toward benign.
2. Feature Contributions
   - Blue bars (supporting benign classification):
     - Worst perimeter (165.90)
     - Mean area (1130.00)
     - Worst area (1866.00)
     - Worst texture (26.58)
   - Orange bars (supporting malignant classification):
     - Mean perimeter (123.60)
     - Worst radius (24.86)
     - Mean radius (18.94)
     - Area error (96.05)
   - Since the blue features dominate, the model predicts benign (0) with full confidence.
3. Decision Splits & Feature Importance
   - The middle section shows decision splits used by the model.
     - For example, “mean perimeter > 105.62” slightly supports malignancy (orange).
     - However, features like “worst perimeter > 125.30” strongly push toward benign.

#### Final Interpretation

Even though a few features (like mean perimeter and worst radius) slightly support malignancy, the overall strong benign indicators (worst perimeter, mean area, worst area) outweigh them completely.
The model is highly confident this tumor is benign (100% probability).


## Model Deployment

The final piece of the Machine Learning modeling puzzle is that of
deploying the model in production so that we actually start using it.


### Persist model to disk

For persisting our model to disk, we can leverage libraries like pickle or joblib, which is also available
with scikit-learn. This allows us to deploy and use the model in the future, without having to retrain it
each time we want to use it.


In [ ]:
# Saves the trained logistic regression model to a file named 'lr_model.pkl' using joblib for later use.
joblib.dump(logistic, "lr_model.pkl")

### Load model from disk

So whenever we will load
this object in memory again we will get the logistic regression model object.


In [ ]:
# Load a pre-trained Logistic Regression model from a pickle file named "lr_model.pkl".
lr = joblib.load("lr_model.pkl")
# Display the loaded Logistic Regression model object. This will show the model's parameters and structure.
lr

### Predict with loaded model

We can now use this lr object, which is our model loaded from the disk, and make predictions.


In [ ]:
# Print the true value from the y_test dataset for the index range 10 to 11 (exclusive of 11, so effectively index 10).
print("True value: ", y_test[10:11])
# Print the predicted value for the corresponding input data from X_test dataset at index range 10 to 11, using the trained linear regression model 'lr'.
print("Predicted value: ", lr.predict(X_test[10:11]))

## Conclusion

Through this tutorial, you have gained practical experience in making machine learning models more transparent and interpretable, optimizing their performance through proper tuning, and preparing them for real-world deployment. These skills are essential for bridging the gap between technical implementation and business understanding in machine learning projects.

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.
